In [1]:
import tensorflow as tf
import datetime
import Models
import AttentionModels
import Common
import VideoDataGenerator
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical

## Prep

In [3]:
# chose split
split_no = 1

### HMDB51 Video Data with 25 Frames

In [4]:
dataset = Common.evaluate_dataset("D:/processed_datasets/hmdb51_vid25/")
split_df = Common.get_hmdb51_split("D:/datasets/hmdb51_org_splits", split_no=split_no)
hmdb51_video_dataset = dataset.merge(split_df, on="filename")
# encode y
hmdb51_video_dataset.category = preprocessing.LabelEncoder().fit_transform(hmdb51_video_dataset.category)
hmdb51_video_dataset

,path,filename,category,split
0,D:\processed_datasets\hmdb51_vid25\brush_hair\...,April_09_brush_hair_u_nm_np1_ba_goo_0.avi,0,1
1,D:\processed_datasets\hmdb51_vid25\brush_hair\...,April_09_brush_hair_u_nm_np1_ba_goo_1.avi,0,1
2,D:\processed_datasets\hmdb51_vid25\brush_hair\...,April_09_brush_hair_u_nm_np1_ba_goo_2.avi,0,1
3,D:\processed_datasets\hmdb51_vid25\brush_hair\...,atempting_to_brush_my_hair_brush_hair_u_nm_np2...,0,1
4,D:\processed_datasets\hmdb51_vid25\brush_hair\...,atempting_to_brush_my_hair_brush_hair_u_nm_np2...,0,1
...,...,...,...,...
6761,D:\processed_datasets\hmdb51_vid25\wave\winKen...,winKen_wave_u_cm_np1_ba_bad_0.avi,50,1
6762,D:\processed_datasets\hmdb51_vid25\wave\winKen...,winKen_wave_u_cm_np1_ri_bad_1.avi,50,1
6763,D:\processed_datasets\hmdb51_vid25\wave\_I_ll_...,_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_M...,50,1
6764,D:\processed_datasets\hmdb51_vid25\wave\_I_ll_...,_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_M...,50,1


### HMDB51 OptFlow Data based on 25 Frames with L=1

In [5]:
dataset = Common.evaluate_dataset("D:/processed_datasets/hmdb51_optflowl1_npz25/")
split_df = Common.get_hmdb51_split("D:/datasets/hmdb51_org_splits", split_no=split_no)
# fix file type extensions
split_df["filename"] = split_df["filename"].str.split(".", expand=True)[0] + ".npz"
hmdb51_optflow_dataset = dataset.merge(split_df, on="filename")
hmdb51_optflow_dataset

,path,filename,category,split
0,D:\processed_datasets\hmdb51_optflowl1_npz25\b...,April_09_brush_hair_u_nm_np1_ba_goo_0.npz,brush_hair,1
1,D:\processed_datasets\hmdb51_optflowl1_npz25\b...,April_09_brush_hair_u_nm_np1_ba_goo_1.npz,brush_hair,1
2,D:\processed_datasets\hmdb51_optflowl1_npz25\b...,April_09_brush_hair_u_nm_np1_ba_goo_2.npz,brush_hair,1
3,D:\processed_datasets\hmdb51_optflowl1_npz25\b...,atempting_to_brush_my_hair_brush_hair_u_nm_np2...,brush_hair,1
4,D:\processed_datasets\hmdb51_optflowl1_npz25\b...,atempting_to_brush_my_hair_brush_hair_u_nm_np2...,brush_hair,1
...,...,...,...,...
6761,D:\processed_datasets\hmdb51_optflowl1_npz25\w...,winKen_wave_u_cm_np1_ba_bad_0.npz,wave,1
6762,D:\processed_datasets\hmdb51_optflowl1_npz25\w...,winKen_wave_u_cm_np1_ri_bad_1.npz,wave,1
6763,D:\processed_datasets\hmdb51_optflowl1_npz25\w...,_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_M...,wave,1
6764,D:\processed_datasets\hmdb51_optflowl1_npz25\w...,_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_M...,wave,1


### prepare VideoDataGenerator

In [6]:
train_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].category.tolist(),
    num_classes = 51,
    optflow_path=hmdb51_optflow_dataset[hmdb51_optflow_dataset.split == 1].path.tolist(),
    batch_size=1,
    target_size=(224, 224),
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input
)

In [7]:
test_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].category.tolist(),
    num_classes = 51,
    optflow_path=hmdb51_optflow_dataset[hmdb51_optflow_dataset.split == 1].path.tolist(),
    batch_size=1,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input
)

## TwoStream-Network (BaseResnet50v2)

In [11]:
twostream_resnet50v2 = Models.create_TwoStreamModel(
        input_shape=(None, 224, 224, 3),
        optflow_shape=(None, 224, 224, 2),
        classes=51
    )

C:\tools\Anaconda3\envs\attention-tf2-gpu\lib\site-packages\tensorflow\python\keras\applications\imagenet_utils.py:331: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  warnings.warn('This model usually expects 1 or 3 input channels. '


In [12]:
log_dir = "logs/fit_twostream_25_L1/resnet50v2_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

twostream_resnet50v2.fit(
    train_gen,
    epochs=1,
    validation_data=test_gen,
    callbacks=[tensorboard_callback]
)

1/1 [==============================] - 0s 4ms/step - loss: 3.5257 - accuracy: 0.0000e+00 - top_k_categorical_accuracy: 0.0000e+00


In [ ]:
twostream_resnet50v2.save("models/twostream_25_L1/ResNet50v2")